# 多个输入通道
彩色图像可能有RGB三个通道
转换为灰度会丢失信息

# 多个输入通道
每个通道都有一个卷积核，结果是所有通道卷积结果的和
 ·输入$\mathbf{X}:c_i\times n_h\times n_w$
 ·核$\mathbf{W}:c_i\times k_h\times k_w$
 ·输出$\mathbf{Y}:m_{h}\times m_{w}$
 $$\mathbf{Y}=\sum_{i=0}^{c_i}\mathbf{X}_{i,:,:}\star\mathbf{W}_{i,:,:}$$

# 多个输出通道
无论有多少输入通道，到目前为止我们只用到单输出通道
我们可以有多个三维卷积核，每个核生成一个输出通道

· 输入$\mathbf{X}:c_i\times n_h\times n_w$
·核$\mathbf{W}:c_{o}\times c_{i}\times k_{h}\times k_{w}$
·输出$\mathbf{Y}:c_o\times m_h$文$m_{w}$
$$\mathbf{Y}_{i,:,:}=\mathbf{X}\star\mathbf{W}_{i,:,:,:}\quad\text{for}i=1,...,c_o$$

# 多个输入和输出通道
·每个输出通道可以识别特定模式
输入通道核识别并组合输入中的模式

# 1x1卷积层
$k_{h}=k_{w}=1$是一个受欢迎的选择。 它不识别空间模式，只是融合通道。
